# Data Cleaning

## Introduction

This notebook goes through a necessary step of any data science project - data cleaning. Data cleaning is a time consuming and unenjoyable task, yet it's a very important one. Keep in mind, "garbage in, garbage out". Feeding dirty data into a model will give us results that are meaningless.

Specifically, we'll be walking through:

1. **Getting the data -** in this case, we'll be scraping data from a website
2. **Cleaning the data -** we will walk through popular text pre-processing techniques
3. **Organizing the data -** we will organize the cleaned data into a way that is easy to input into other algorithms

The output of this notebook will be clean, organized data in two standard text formats:

1. **Corpus** - a collection of text
2. **Document-Term Matrix** - word counts in matrix format

## Problem Statement

As a reminder, our goal is to look at transcripts of various comedians and note their similarities and differences. Specifically, I'd like to know if Ali Wong's comedy style is different than other comedians, since she's the comedian that got me interested in stand up comedy.

## Getting The Data

Luckily, there are wonderful people online that keep track of stand up routine transcripts. [Scraps From The Loft](http://scrapsfromtheloft.com) makes them available for non-profit and educational purposes.

To decide which comedians to look into, I went on IMDB and looked specifically at comedy specials that were released in the past 5 years. To narrow it down further, I looked only at those with greater than a 7.5/10 rating and more than 2000 votes. If a comedian had multiple specials that fit those requirements, I would pick the most highly rated one. I ended up with a dozen comedy specials.

In [44]:
# Web scraping, pickle imports
import os
import requests
from bs4 import BeautifulSoup
import pickle

# Scrapes transcript data from scrapsfromtheloft.com
# def url_to_transcript(url):
def url_to_transcript(url):
    '''
    This function get igihe article content and save the content on the txt file
    '''
    '''Returns transcript data specifically from http://m.igihe.com/'''
    page =  requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    try :
        article =  soup.find_all('div', class_="inkuru-title-2")[0]
    except IndexError as e:
        print('Another IndexError: ', e)
        return 
    paragraphs = article.find_all('p')
    filename = url.rsplit('/', 1)[-1] + '.txt'

    with open(os.path.join(filename), 'w+') as f:
        for para in paragraphs:
            f.write(para.getText()+ '\n \n')

# URLs of transcripts in scope
urls = ['http://m.igihe.com/imyidagaduro/article/nsengiyumva-igisupusupu-yasohoye-indirimbo-nshya-umutesi',
        'http://m.igihe.com/amakuru/mu-mahanga/article/kiliziya-yahinduye-amabwiriza-agenga-ikurikiranwa-ry-abihayimana-bakekwaho',
        'http://m.igihe.com/amakuru/muri-afurika/article/sudani-y-epfo-kiir-na-machar-bemeranyije-gushinga-guverinoma-y-ubumwe',
        'http://m.igihe.com/imikino/football/article/apr-fc-yahize-rayon-sports-mu-ikipe-y-umwaka-wa-2019-sunrise-fc-ihagararira',
        'https://mobile.igihe.com/ubukungu/ishoramari/article/inyungu-u-rwanda-rwiteze-mu-imurikagurisha-ry-ibikorerwa-muri-oman-rugiye',
        'http://m.igihe.com/https://mobile.igihe.com/ubukungu/ishoramari/article/inyungu-u-rwanda-rwiteze-mu-imurikagurisha-ry-ibikorerwa-muri-oman-rugiye',
        'http://m.igihe.com/amakuru/u-rwanda/article/nyamasheke-ibikorwaremezo-bike-ku-kiyaga-cya-kivu-imbogamizi-ku-iterambere',
        'http://m.igihe.com/imyidagaduro/article/imbamutima-z-abatsinze-muri-hanga-higa-basabye-alain-muku-ko-yabatuza-i-kigali',
        'http://m.igihe.com/amakuru/mu-mahanga/article/pervez-musharraf-wahoze-ayobora-pakistan-yakatiwe-igihano-cy-urupfu',
        'http://m.igihe.com/ubukungu/article/abanyarwanda-bishimiye-icyemezo-cya-guverinoma-cyo-kuvanaho-tva-ku-bikoresho-by']

# Article names (comedians=articles)
articles = ['article1', 'article2', 'article3', 'article4', 'article5', 'article6', 'article7', 'article8', 'article9', 'article10']

In [45]:
# # Actually request transcripts (takes a few minutes to run)
transcripts = [url_to_transcript(u) for u in urls]

Another IndexError:  list index out of range


In [81]:
pwd

'/home/desire/Documents/programming/github/kinyarwandaTexts'

In [82]:
import os
import json
import nltk
from nltk.tokenize import word_tokenize
import re
#stopwods : useless words ( uhmm, amm)
#from nltk.corpus import stopwords
#nltk.download('stopwords')
#rootdir = 'C:/Users/Sababu/Desktop/Kinyarwanda/Digital_Umuganda_data/words
rootdir = '/home/desire/Documents/programming/github/kinyarwandaTexts/DATA/texts'
db =[]

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".txt"):
            rd = open(filepath, encoding="ISO-8859-1").read().lower()
            wrd = word_tokenize(rd)
            words = [word for word in wrd if word.isalpha()]   
                    
            neww = words
            db.extend(neww)
 

In [84]:
def remove(list): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list] 
    return list  

new_file=remove(db) 
count = list(set(db)) 

print(len(db)) #all words in all .txt files including common

62803


In [85]:
#remove special characters & numbers
new_filee = ''.join(e for e in new_file if e.isalnum()) 
print(len(new_filee))
print(len(count))# unique words


366200
12255


In [86]:
#putting them into one file

with open("words.txt", "w+") as filehandle:
    json.dump(count, filehandle,indent=2)


    print('DONE')

DONE


In [98]:
with open("duplicates.txt", "w+") as filehandle:
    json.dump(new_file, filehandle,indent=2)

In [95]:
print(count[2:12])

['muvuduko', 'gihangayikishije', 'kubicuranga', 'ihembwa', 'bashishikariza', 'kanobana', 'kisoro', 'gufasha', 'urubyiniro', 'ntoki']


## Cleaning The Data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Here are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.

**Common data cleaning steps on all text:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words

**More data cleaning steps after tokenization:**
* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more...

In [ ]:
# Let's take a look at our data again
next(iter(data.keys()))

In [ ]:
# Notice that our dictionary is currently in key: comedian, value: list of text format
next(iter(data.values()))

In [ ]:
# We are going to change this to key: comedian, value: string format
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [ ]:
# Combine it!
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [125]:
#import pandas as pd
data = pd.read_csv('words.txt', skiprows=1,  sep=",", header=None,  names=["words", "col"])
#data.columns =  ["a", "b"]

In [127]:
del data['col']

data[1:6]

,words
1,"""rishake"""
2,"""muvuduko"""
3,"""gihangayikishije"""
4,"""kubicuranga"""
5,"""ihembwa"""


In [129]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd

data_df = pd.DataFrame.from_dict(data).transpose()
data_df = pd.DataFrame.from_dict(data).transpose()
#data_df.columns = ['words']
data_df = data_df.sort_index()

data_df

,0,1,2,3,4,5,6,7,8,9,...,12246,12247,12248,12249,12250,12251,12252,12253,12254,12255
words,"""umucyo""","""rishake""","""muvuduko""","""gihangayikishije""","""kubicuranga""","""ihembwa""","""bashishikariza""","""kanobana""","""kisoro""","""gufasha""",...,"""banki""","""byinjiza""","""bategura""","""awards""","""bahita""","""twahakekaga""","""ugabanuka""","""bugenzacyaha""","""bakanagirana""",]


In [133]:
df = pd.DataFrame.from_dict(data)
df = df.sort_index()

df

,words
0,"""umucyo"""
1,"""rishake"""
2,"""muvuduko"""
3,"""gihangayikishije"""
4,"""kubicuranga"""
...,...
12251,"""twahakekaga"""
12252,"""ugabanuka"""
12253,"""bugenzacyaha"""
12254,"""bakanagirana"""


In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_clean

**NOTE:** This data cleaning aka text pre-processing step could go on for a while, but we are going to stop for now. After going through some analysis techniques, if you see that the results don't make sense or could be improved, you can come back and make more edits such as:
* Mark 'cheering' and 'cheer' as the same word (stemming / lemmatization)
* Combine 'thank you' into one term (bi-grams)
* And a lot more...

## Organizing The Data

I mentioned earlier that the output of this notebook will be clean, organized data in two standard text formats:
1. **Corpus - **a collection of text
2. **Document-Term Matrix - **word counts in matrix format

### Corpus

We already created a corpus in an earlier step. The definition of a corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.

In [ ]:
# Let's take a look at our dataframe
data_df

In [130]:
# Let's pickle it for later use
data_df.to_pickle("corpus.pkl")

In [134]:
# Let's pickle it for later use
df.to_pickle("corpus_df.pkl")

### Document-Term Matrix

For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text.

## Additional Exercises

1. Add an additional regular expression to further clean the text
2. Play around with CountVectorizer's parameters. ngram_range? min_df and max_df?